In [14]:
# Funtkion um Alle Daten herunterzuladen

In [15]:
# Module Imports

#DB stuff
import mariadb
import sys

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time
import datetime

In [16]:
#Connect with DB
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [17]:
########################
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]

def pad(data):
  length = 16 - (len(data) % 16)
  return data + chr(length).encode('utf-8')*length

def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

def encodeMessage(msg):
  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Erster Schritt: AES256 verschlüsseln
  iv = Random.new().read(AES.block_size)
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
  msg = msg.encode('utf-8')
  crypt = iv + cipher.encrypt(pad(msg))

  # Zweiter Schritt: base64 enkodieren
  msgb64 = base64.b64encode(crypt).decode('utf-8')
  return msgb64

#Gibt ein JSON-Objekt über die verfügbaren Daten zurück
def getAvailable():
    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Daten anfordern
    connection.request("GET", "/dirbuff")
    contents = connection.getresponse()

    # Daten entschlüsseln
    msg = decodeMessage(contents.read())
  
    #JSON-String umwandeln in dict
    folder_dict = json.loads(msg)
    
    # Verbindung trennen
    connection.close()

    return folder_dict

#Funktion zur erstellung des SQL-Statements
def sql_data(contents, cur):
  #content wird Zeile für Zeile verarbeitet
  for line in contents.read().split(b'\n'):
      if line != b'':
          #Message wird dekodiert
          line = decodeMessage(line)
          print(line)

          #Zeile wird in ein dict konvertiert
          line= json.loads(line)

          columns = ""
          values = ""

          if line["Status"] != "OK":
                  print(line["Status"])
                  print("Skipped invalid measurements due to warm-up of the Sensor")
                  continue
          
          for type in line:
              #Diese Daten werden herausgefiltert und nicht mit in die Datenbank mit aufgenommen
              if type == "bat" or type == "DeviceID" or type == "uptime" or type == "window_event" or type == "door_event" or type == "person" or type == "window_open" or type == "Status":
                continue

              
              #Erstellt den String für die Columns
              columns += type 

              #Erstellt den String für die Values 
              #Edge-case: wenn value eine Liste ist
              if isinstance(line[type], list):
                  values += str(line[type][0])
              elif type == "timestamp":
                  values += "FROM_UNIXTIME(%s)" % (int(line[type]/1000))
              else:  
                  values += str(line[type])

              #Edge-case: Am Ende kein Komma
              if type != "cnt0_3":  
                  columns += ", "
                  values += ", "

          sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s );" % ('measurements', columns, values)
          print(sql)
          cur.execute(sql)
          conn.commit()

In [18]:
def file_to_db():
    folder = getAvailable()

    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)
    

    for year in folder.keys():
        for month in folder[year].keys():
            for day in folder[year][month].keys():
                for file in folder[year][month][day]:
                    # Anfrage formulieren und Daten anfordern
                    print("_____________________________________________________________________________")
                    print("Year: " + year + "\n" + "Month: " + month + "\n" + "Day: " + day + "\n" + "File: " + file + "\n")   
                    connection.request("GET","/file?request="+encodeMessage(year+"/"+month+"/"+day+"/"+file))
                    contents = connection.getresponse()
                    if contents.status == 200:
                        print("Status: " + str(contents.status) + "/OK")
                    else:
                        print("Status: " + str(contents.status) + "/Canceled!")
                        break
                    sql_data(contents, cur)

    # Verbindung trennen
    connection.close()

In [19]:
print(getAvailable())

{'2023': {'5': {'25': ['1684972740', '1684975998', '1684979252', '1684982517', '1684985779', '1684989047', '1684992323', '1684995593', '1684998853', '1685002120', '1685005383', '1685008644', '1685011909', '1685015161', '1685018420', '1685021675', '1685024932', '1685028193', '1685031452', '1685034706', '1685037964', '1685041217', '1685044474', '1685047740', '1685051004', '1685054265', '1685057529'], '3': ['1683071974', '1683075222', '1683078472', '1683081719', '1683084969', '1683088216', '1683091463', '1683094712', '1683097960', '1683101210', '1683104460', '1683107708', '1683110957', '1683114205', '1683117455', '1683120701', '1683123948', '1683127238', '1683130526', '1683133809', '1683137141', '1683140388', '1683143638', '1683146888', '1683150135', '1683153382', '1683156629'], '5': ['1683244752', '1683248002', '1683251248', '1683254499', '1683257746', '1683260994', '1683264243', '1683267491', '1683270740', '1683273988', '1683277234', '1683280481', '1683283729', '1683286977', '1683290226

In [20]:
#DB-Connection close
conn.close()